## Day 3
Url: https://adventofcode.com/2022/day/3

Problem: each line of the input (`'./inputs/day3.txt'`) contains the inventory of items in a elf's backpack. The first half of the line is in one compartment of the backpack, the second half is in a second compartment. The elf who packed the items did a mistake and packed one item in both compartments for every elf. 

We need to find:
1) which was the misplaced element for each elf, and
2) assign to each misplaced element a score (given by the problem)
3) calculate the total score of all the misplaced elements

Scoring:
a to z: 1 to 26
A to Z: 27 to 52


In [1]:
import pandas as pd
import numpy as np
import string

In [2]:
inventory = pd.read_csv('./inputs/day3.txt', header=None)
inventory.columns = ['list_of_items']
inventory['misplaced_item'] = None
inventory['assigned score'] = 0

In [3]:
inventory

,list_of_items,misplaced_item,assigned score
0,LdHVLDLDdHdtLMhcqCqGWcWg,None,0
1,ZZQZSZnnJrQrJQJbfzfnWGWPWMcChMMPcqMnhFcF,None,0
2,ZrzpWzfbpQpWbzvZWZpdtVtDNmRHNVptNDHt,None,0
3,gzCjffWZCtCfZZVdqVSqJdvJndSt,None,0
4,hMHLcmGLMLhHmsRMsSvsQSqrsrlJTTdV,None,0
...,...,...,...
295,DLWRLjRrmNPQjZZlQPsFGFggVcWcFddggdsg,None,0
296,rjrZPwwDRlLLBjQlRRlPDpmbqzpqnnCSCfTMwMqSvC,None,0
297,FmcGcjLRPjQwQjMQrwHQ,None,0
298,btJzJbVNdBJJtzTdGBbdBztGrQhhQWhMwHrhrHSHgHQfhMVS,None,0


We write a couple of functions to find and calculate the value of the item:

In [63]:
def assign_value_to_item(item):
    point_mapping = pd.DataFrame({'letters': list(string.ascii_lowercase + string.ascii_uppercase),
              'values' : range(1, 53)})
    matching_value = point_mapping.loc[point_mapping['letters'] == item]['values'].values

    return matching_value

def find_misplaced_item(list_of_items):
    number_of_items = len(list_of_items)
    break_index = int(number_of_items/2)
    in_first_compartment = list_of_items[:break_index]
    in_second_compartment = list_of_items[break_index:]
    assert(len(in_first_compartment) == len(in_second_compartment))

    # use set() to find commonalities:
    common_item = list( set(in_first_compartment) & set(in_second_compartment) )
    
    return common_item[0]


In [58]:

test = inventory['list_of_items'][0]
length = int(len(test)/2)
test[:length]


'LdHVLDLDdHdt'

Now we assign this function column wise:

In [66]:
for i in range(0, len(inventory)):
    inventory['misplaced_item'][i] = find_misplaced_item(inventory['list_of_items'][i])
    inventory['assigned score'][i] = assign_value_to_item(inventory['misplaced_item'][i])
    

/tmp/ipykernel_400734/1939551171.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inventory['misplaced_item'][i] = find_misplaced_item(inventory['list_of_items'][i])
/tmp/ipykernel_400734/1939551171.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inventory['assigned score'][i] = assign_value_to_item(inventory['misplaced_item'][i])


In [67]:
inventory

,list_of_items,misplaced_item,assigned score
0,LdHVLDLDdHdtLMhcqCqGWcWg,L,38
1,ZZQZSZnnJrQrJQJbfzfnWGWPWMcChMMPcqMnhFcF,n,14
2,ZrzpWzfbpQpWbzvZWZpdtVtDNmRHNVptNDHt,p,16
3,gzCjffWZCtCfZZVdqVSqJdvJndSt,t,20
4,hMHLcmGLMLhHmsRMsSvsQSqrsrlJTTdV,s,19
...,...,...,...
295,DLWRLjRrmNPQjZZlQPsFGFggVcWcFddggdsg,W,49
296,rjrZPwwDRlLLBjQlRRlPDpmbqzpqnnCSCfTMwMqSvC,w,23
297,FmcGcjLRPjQwQjMQrwHQ,j,10
298,btJzJbVNdBJJtzTdGBbdBztGrQhhQWhMwHrhrHSHgHQfhMVS,V,48
